In [7]:
import praw
import pandas as pd
from datetime import datetime

# Set up Reddit API credentials
client_id = 'Bkulv7ePr58Vu37vLBObAg'
client_secret = 'jJTaqSbHFW-BXSYh0Br4copXSsiSWw'
user_agent = 'Mozilla/5.0'

# Initialize the PRAW Reddit instance
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Define the Reddit post URL
post_url = 'https://www.reddit.com/r/NTU/comments/1fccw77/'  # Replace 'post_id' with the actual ID of the post

# Get the submission (post) object
submission = reddit.submission(url=post_url)

# Convert the UTC timestamp to a human-readable format
post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')

# Initialize a list to store post details and comments
comments_dataset = []

# Extract submission (post) details
post_details = {
    'post_id': submission.id,
    'post_title': submission.title,
    'post_author': submission.author.name if submission.author else 'deleted',
    'post_author_flair': submission.author_flair_text,  # Extract the author's flair
    'post_body': submission.selftext,
    'post_score': submission.score,
    'post_url': submission.url,
    'post_num_comments': submission.num_comments,
    'post_date': post_date  # Add post date
}

# Function to recursively extract comments and replies
def extract_comments(comment, parent_id=None):
    # Add the current comment to the dataset
    comments_dataset.append({
        'comment_id': comment.id,
        'parent_id': parent_id,
        'comment_author': comment.author.name if comment.author else 'deleted',
        'comment_author_id': comment.author.id if comment.author else 'deleted',  # Extract the comment author's user_id
        'comment_author_flair': comment.author_flair_text,  # Extract the comment author's flair
        'comment_body': comment.body,
        'comment_score': comment.score
    })
    
    # Recursively extract replies
    for reply in comment.replies:
        extract_comments(reply, parent_id=comment.id)

# Fetch all top-level comments and extract their replies
submission.comments.replace_more(limit=None)  # Ensure all comments are loaded
for top_level_comment in submission.comments.list():
    extract_comments(top_level_comment)

# Convert the comments dataset to a DataFrame
df_comments = pd.DataFrame(comments_dataset)

# Add post details to each comment (so each row contains both post and comment info)
df_comments['post_id'] = post_details['post_id']
df_comments['post_title'] = post_details['post_title']
df_comments['post_author'] = post_details['post_author']
df_comments['post_author_id'] = post_details['post_author_id']
df_comments['post_author_flair'] = post_details['post_author_flair']
df_comments['post_body'] = post_details['post_body']
df_comments['post_score'] = post_details['post_score']
df_comments['post_url'] = post_details['post_url']
df_comments['post_num_comments'] = post_details['post_num_comments']
df_comments['post_date'] = post_details['post_date']  # Add post date to each row

# Save the comments and post data to a CSV file
df_comments.to_csv('test_dataset.csv', index=False)

print("Post and comments with flair and post date extracted and saved to 'test_dataset.csv'.")


Post and comments with flair and post date extracted and saved to 'test_dataset.csv'.


C:\Users\Chris\AppData\Local\Temp\ipykernel_20752\4026838166.py:24: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
